# 8. Content-based filtering

In [1]:
%run "01.Recommendation_Loading.ipynb"

################  Recommendations (all data, positive recommendations only) ################
Users: 12,636,209
Games: 37,419
Number of users 10,000
First five users id [11203022 12827342 11719330  9553563 11051045]
################  Recommendations (Sample) ################
Minimum number of recommendations to included in the sample: 20
Number of randomly selected users: 10000
Users: 9,688
Games: 20,615
################ User-Game Matrix ################
Number of Rows: 9,689
Numbeer of Columns: 20,616
Number of stored values: 365,753


In [2]:
matrix_games_tags.info()

NameError: name 'matrix_games_tags' is not defined

### Dimensionality Reduction

Principal Component Analysis is conducted to reduce the number of game tags.

In [ ]:
pca = PCA(n_components=len(matrix_games_tags.columns))
pca.fit(matrix_games_tags)
cumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)

In [ ]:
# Finding the suitable value of k
k = np.argmax(cumulative_variance_ratio >= 0.95) + 1

print("The suitable value of k for PCA with a threshold of 0.95 is:", k)

# View cumulative explained variance ratio with threshold
plt.plot(range(1, len(cumulative_variance_ratio) + 1), cumulative_variance_ratio, marker='o')
plt.axhline(y=0.95, color='r', linestyle='--', label='Threshold: 0.95')
plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.title('Cumulative Explained Variance Ratio by Number of Principal Components')
plt.grid(True)
plt.show()

In [ ]:
N = 10
k = 250
# Dimensionality Reduction: Apply PCA
pca = PCA(n_components=k)  # Set the desired number of components, k
reduced_features = pca.fit_transform(matrix_games_tags)

# Convert the reduced_features array back to a DataFrame
reduced_matrix_games_tags = pd.DataFrame(reduced_features, index=matrix_games_tags.index)

# Content-based Filtering (Similarity)
# Calculate similarity matrix
similarity_matrix = cosine_similarity(reduced_matrix_games_tags)

In [ ]:
# Example on one game
selected_game_id = 12345  # Choose the game ID for which you want to find similar games
similarity_scores = similarity_matrix[selected_game_id]
similar_games_indices = similarity_scores.argsort()[::-1][:N]  # Select top N similar games

recommended_games = matrix_games_tags.iloc[similar_games_indices].index.tolist()